In [ ]:
%load_ext autoreload
%autoreload 2
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from model.model import load_trained_model

model = load_trained_model('epfl_picasso_retrain_2')
print(model.summary())

In [ ]:
all_vals_true = []
all_vals_pred = []

In [ ]:
'''
--------
|    x |   <- 50nm deeper than other
|      |
| x    |
--------

5 structures, frame 16 px,
structureX: 1000,3000
structureY: 1000,3000

structure3D: 0,50
ExchangeLabels:1,1
'''

# dirpath = '/home/miguel/Projects/uni/data/smlm_3d/picasso_test/nanopore/NPC-A647-3D-stack_1'

# dirpath = '/home/miguel/Projects/uni/data/smlm_3d/picasso_sim/500nm/500nm'
# dirpath = '/home/miguel/Projects/uni/data/smlm_3d/picasso_sim/50nm/50nm'

dirpath = '/home/miguel/Projects/uni/data/smlm_3d/picasso_sim/nanopore/nanopore_epfl_50nm'

# true_cluster_count = 1
# dirpath = '/home/miguel/Projects/uni/data/smlm_3d/picasso_sim/grid_single/grid_single'
# true_cluster_count = 2
# dirpath = '/home/miguel/Projects/uni/data/smlm_3d/picasso_sim/grid_pairs/grid_pairs'
# true_cluster_count = 3
# dirpath = '/home/miguel/Projects/uni/data/smlm_3d/picasso_sim/grid_triples/grid_triples_more_frames'

# dirpath = '/home/miguel/Projects/uni/data/smlm_3d/picasso_sim/step_grid/step_grid'

# tubules
# dirpath = '/home/miguel/Projects/uni/data/smlm_3d/epfl/tubules/low_density/sequence-as-stack-MT0.N1.LD-AS-Exp'

In [ ]:
import yaml

yaml_file = f'{dirpath}.yaml'

class SafeLoaderIgnoreUnknown(yaml.SafeLoader):
    def ignore_unknown(self, node):
        return None 

SafeLoaderIgnoreUnknown.add_constructor(None, SafeLoaderIgnoreUnknown.ignore_unknown)

with open(yaml_file, "r") as stream:
    root = yaml.load(stream, Loader=SafeLoaderIgnoreUnknown)
with open(yaml_file, "w") as stream:
    yaml.dump(root, stream)

In [ ]:
%load_ext autoreload
%autoreload 2

import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = [5,5]

locs = f'{dirpath}_locs.hdf5'
spots = f'{dirpath}_spots.hdf5'
df = pd.read_hdf(locs, 'locs')



f = h5py.File(spots, 'r')
spots = np.array(f['spots'])[:, :, :, np.newaxis]
f.close()
 
# df['id'] = np.arange(0, df.shape[0])
# df = df.loc[(df['x'] > 20) &(df['x'] < 60) & (df['y'] > 20) &(df['y'] < 60)]
# spots = spots[list(df['id'].to_numpy())]

def norm_psfs(psfs):
    psfs_max = psfs.max(axis=(1,2,3))[:, np.newaxis, np.newaxis, np.newaxis]
    psfs_min = psfs.min(axis=(1,2,3))[:, np.newaxis, np.newaxis, np.newaxis]
    return (psfs - psfs_min) / (psfs_max - psfs_min)

print(spots.shape)
n_spots = 100000
spots = spots[:n_spots]
df = df.iloc[:n_spots]
print(spots.shape)
import tensorflow as tf
from skimage.transform import resize


n_channels = model.input[0].shape[-1]
if n_channels == 3:
    spots = tf.image.grayscale_to_rgb(tf.convert_to_tensor(spots)).numpy()

spots = np.stack([resize(img, (32, 32, spots.shape[-1]), anti_aliasing=True) for img in spots])

spots = norm_psfs(spots)
print(spots.min(), spots.max())
print(spots.shape)
print(df.shape)

In [ ]:
# # Rescale to [-1, 1]
# spots = np.stack([norm_one_one(img) for img in spots])


plt.gca().invert_yaxis()

df['index'] = np.arange(0, df.shape[0])
print(spots.shape)
plt.rcParams['figure.figsize'] = [5, 5]
sns.scatterplot(data=df, x='x', y='y', marker='+')
plt.show()


coords = np.zeros((spots.shape[0], 2))

preds = model.predict((spots, coords)).squeeze()
print(np.std(preds))
df['pred'] = preds

print(preds.min(), preds.max())

plt.rcParams['figure.figsize'] = [5, 5]
sns.histplot(preds, bins=50)
plt.xlabel('Z position (nm)')
plt.show()


n_psfs = 10000
sub_imgs = spots[:n_psfs]
sub_preds = preds[:n_psfs]
plt.rcParams['figure.figsize'] = [100, 100]
from data.visualise import grid_psfs
print(np.sort(sub_preds.squeeze()))

plt.imshow(grid_psfs(sub_imgs[np.argsort(sub_preds.squeeze())].mean(axis=-1).squeeze()))
plt.show()

In [ ]:
from data.visualise import scatter_3d, scatter_yz
plt.rcParams['figure.figsize'] = [15, 15]
zxy = np.concatenate((df[['x', 'y']].to_numpy(), preds[:, np.newaxis]), axis=1)
scatter_3d(zxy)
scatter_yz(zxy)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import DBSCAN

plt.rcParams['figure.figsize'] = [5,5]

coords = df[['x', 'y']].to_numpy() * 100
# distance_matrix = euclidean_distances(coords, coords)
eps = 5
min_count = 50
# distance_matrix = (distance_matrix < eps).astype(int).sum(axis=0)
# idx = np.argwhere(distance_matrix > min_count).squeeze()
# sub_coords = coords[idx]
# print(coords.shape)
# print(sub_coords.shape)

sub_coords = coords
cluster_ids = DBSCAN(eps=eps, min_samples=min_count).fit_predict(coords)
print(len(set(cluster_ids)))
sns.scatterplot(x=sub_coords[:, 0], y=sub_coords[:, 1], hue=cluster_ids.astype(str), legend=False)
plt.axis('equal')
plt.gca().invert_yaxis()
plt.show()

sns.kdeplot(data=df, x='x', y='y')
plt.show()

df['cluster_id'] = cluster_ids.squeeze()
idx = (df['cluster_id']!=-1).to_numpy()
sub_spots = spots[idx]
sub_df = df.iloc[idx]
sub_preds = preds[idx]
# sub_df = df.iloc[idx]
# sub_df['cluster_id'] = cluster_ids.squeeze().astype(str)
# sub_spots = spots[idx]

In [ ]:
%load_ext autoreload
%autoreload 2
from termcolor import colored

from sklearn.cluster import DBSCAN, OPTICS, KMeans
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
from data.visualise import grid_psfs, show_psf_axial
from tqdm import tqdm

plt.rcParams['figure.figsize'] = [20, 5]
# df['cluster_id'] = OPTICS().fit_predict(df[['x', 'y']].to_numpy()).squeeze().astype(str)
plt.axis('equal')
plt.title('Units: pixels (90nm each)')
sns.scatterplot(data=sub_df, x='x', y='y', marker='.', hue='cluster_id', legend=False)
plt.show()

gm_mean_diffs = {}
mean_img_diffs = {}

VIEW_FIT = True
def gm_min_bic(data, imgs):
    gm_df = pd.DataFrame({'pred': data.squeeze()}, index=np.arange(0, data.squeeze().shape[0]))

    best_gm = None
    min_bic = np.inf
    bics = []
    cov_type = 'tied'
    stdevs = []
    idx = np.argsort(data.squeeze())
    
    if VIEW_FIT:
        print(imgs.shape)
        plt.imshow(grid_psfs(imgs.mean(axis=-1)[idx]))
        plt.show()
        fig, axes = plt.subplots(1, 6)
    for n in range(1, 7):
        gm = GaussianMixture(n_components=n, n_init=20, covariance_type=cov_type).fit(data)
        bic = gm.bic(data)
        
        bics.append(round(bic, 3))
        weight_range = gm.weights_.min() / gm.weights_.max()
        print(weight_range)
        if bic < min_bic and weight_range > 0.3:
            min_bic = bic
            best_gm = gm
        
        labels = gm.predict(data).squeeze()

        gm_df['cluster_id'] = labels.astype(str)

        weights = gm.weights_

        if VIEW_FIT:
            ax = axes[n-1]
            sns.histplot(data=gm_df, x='pred', hue='cluster_id', stat='density', alpha=0.2, bins=20, ax=ax)

            # create necessary things to plot
            x_axis = np.linspace(data.min(), data.max(), 50)
            ys = []
            sub_df2 = pd.DataFrame.from_dict({'x': x_axis})
            for i in range(0, gm.n_components):
                if cov_type == 'tied':
                    cov = gm.covariances_.squeeze()
                elif cov_type == 'full' or cov_type == None:
                    cov = gm.covariances_[i][0][0]
                elif cov_type == 'spherical':
                    cov = gm.covariances_[i]
                elif cov_type == 'diag':
                    cov_type = gm.covariances_[i]

                sub_df2[f'y_{i}'] = norm.pdf(x_axis, float(gm.means_[i][0]), np.sqrt(cov))*gm.weights_[i]
                sns.lineplot(data=sub_df2, x='x', y=f'y_{i}', ax=ax)
    if VIEW_FIT:
        plt.show()        
        
    print(bics)
    print(f'Best EM: {best_gm.n_components}')

    return best_gm.means_[:, 0]

def apply_gm(data, imgs, cid):
    data = data.reshape(-1, 1)
    gm_df = pd.DataFrame({'pred': list(data)})
    
    return gm_min_bic(data, imgs)
    

n_emitters = []
all_coords = []
for cid in tqdm(set(sub_df['cluster_id'])):
    if cid == '-1':
        continue
    idx = np.argwhere(sub_df['cluster_id'].to_numpy()==cid).squeeze()
    imgs = sub_spots[idx]
    coords = np.zeros((imgs.shape[0], 2))
    preds = sub_preds[idx]
#     preds -= preds.min()
#     preds += 0.00000001
#     preds = np.sqrt(preds)
    z_pos = apply_gm(preds, imgs, cid)
    print(np.diff(sorted(z_pos)))
    n_emitters.append(len(z_pos))
    x, y = sub_df.iloc[idx][['x', 'y']].mean(axis=0).to_numpy() * 106
    coords = [[x, y, z, int(cid)] for z in z_pos]
    all_coords.extend(coords)
    
from collections import Counter
print(Counter(n_emitters))
all_coords = np.array(all_coords)
res = pd.DataFrame.from_dict({
    k: all_coords[:, i] for k, i in zip(['x', 'y', 'z', 'cluster_id'], [0, 1, 2, 3])
})


In [ ]:
from data.visualise import scatter_3d, scatter_yz

coords = res[['x', 'y', 'z']].to_numpy()
scatter_3d(coords)

import numpy as np

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

xy = coords[:, 0:2]
z = coords[:, 2]

rhos = list(map(lambda x: cart2pol(x[0], x[1])[0], xy))
plt.scatter(rhos, z)
plt.ylabel('z (nm)')
plt.show()

In [ ]:
all_vals_pred.extend(n_emitters)
all_vals_true.extend([true_cluster_count for _ in n_emitters])
print(len(all_vals_true))
print(len(all_vals_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(all_vals_true, all_vals_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=sorted(set(all_vals_true).union(set(all_vals_pred))))
disp.plot()
plt.show()